### Solution for https://gym.openai.com/envs/MountainCarContinuous-v0/

In [1]:
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1

import gym
import numpy as np
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display

In [2]:
display = Display(visible=0, size=(400, 300))
display.start()

In [3]:
import os
from gym import wrappers
from gym import envs


from IPython import display


import random
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [4]:
env = gym.make('MountainCar-v0')
env.reset()
goal_steps = 200
score_requirement = -198
intial_games = 10000

In [5]:
!mkdir dumbcar

In [ ]:
def play_a_random_game_first():
  img = plt.imshow(env.render('rgb_array')) 
  for step_index in range(goal_steps):
    #env.render()
    img.set_data(env.render('rgb_array')) # just update the data
    display.display(plt.gcf())
    display.clear_output(wait=True)
    plt.savefig('/content/dumbcar/{}.png'.format(step_index))
    action = env.action_space.sample()

    observation, reward, done, info = env.step(action)
    
    print("Step {}:".format(step_index))
    print("action: {}".format(action))
    print("observation: {}".format(observation))
    print("reward: {}".format(reward))
    print("done: {}".format(done))
    print("info: {}".format(info))
    if done:
        break
    env.reset()

In [7]:
if it doesn't work restart runtime.
play_a_random_game_first()

In [ ]:
from PIL import Image, ImageDraw
images = []
for i in os.listdir('/content/dumbcar/'):
  img = Image.open('/content/dumbcar/' + i)
  images.append(img)

images[0].save('dumbcar.gif',save_all=True, append_images=images[1:], optimize=False, duration = 180)

In [9]:
def model_data_preparation():
    training_data = []
    accepted_scores = []
    for game_index in range(intial_games):
        score = 0
        game_memory = []
        previous_observation = []
        for step_index in range(goal_steps):
            action = random.randrange(0, 3)
            observation, reward, done, info = env.step(action)
            
            if len(previous_observation) > 0:
                game_memory.append([previous_observation, action])
                
            previous_observation = observation
            if observation[0] > -0.2:
                reward = 1
            
            score += reward
            if done:
                break
            
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                if data[1] == 1:
                    output = [0, 1, 0]
                elif data[1] == 0:
                    output = [1, 0, 0]
                elif data[1] == 2:
                    output = [0, 0, 1]
                training_data.append([data[0], output])
        
        env.reset()
    
    print(accepted_scores)
    
    return training_data

In [10]:
training_data = model_data_preparation()

[-188.0, -178.0, -198.0, -178.0, -190.0, -170.0, -190.0, -188.0, -166.0, -198.0, -166.0, -184.0, -176.0, -196.0, -180.0, -194.0, -178.0, -184.0, -192.0, -192.0, -196.0, -188.0, -190.0, -194.0, -178.0, -188.0, -180.0, -182.0, -198.0, -190.0, -180.0, -176.0, -188.0, -190.0, -190.0, -168.0]


In [11]:
def build_model(input_size, output_size):
        model = Sequential()
        model.add(Dense(128, input_dim=input_size, activation='relu'))
        model.add(Dense(52, activation='relu'))
        model.add(Dense(output_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam())

        return model

In [12]:
def train_model(training_data):
    X = np.array([i[0] for i in training_data]).reshape(-1, len(training_data[0][0]))
    y = np.array([i[1] for i in training_data]).reshape(-1, len(training_data[0][1]))
    model = build_model(input_size=len(X[0]), output_size=len(y[0]))
    
    model.fit(X, y, epochs=5)
    return model

In [13]:
trained_model = train_model(training_data)

Epoch 1/5
224/224 [==============================] - 3s 2ms/step - loss: 0.2454
Epoch 2/5
224/224 [==============================] - 0s 2ms/step - loss: 0.2228
Epoch 3/5
224/224 [==============================] - 0s 2ms/step - loss: 0.2217
Epoch 4/5
224/224 [==============================] - 0s 2ms/step - loss: 0.2213
Epoch 5/5
224/224 [==============================] - 0s 2ms/step - loss: 0.2207


In [14]:
!mkdir smartcar

If the runtime is crashed comment the part where dumbcar folder and its gif is created, then factory reset and run all

In [ ]:
scores = []
choices = []
c=0
for each_game in range(100):
  score = 0
  game_memory = []
  prev_obs = []
  img = plt.imshow(env.render('rgb_array')) 
  for step_index in range(goal_steps):
    
    img.set_data(env.render('rgb_array')) # just update the data
    display.display(plt.gcf())
    plt.savefig('/content/smartcar/{}.png'.format(step_index)) 
    display.clear_output(wait=True)
    #env.render()
    if len(prev_obs)==0:
        action = random.randrange(0,2)
    else:
        action = np.argmax(trained_model.predict(prev_obs.reshape(-1, len(prev_obs)))[0])
    
    choices.append(action)
    new_observation, reward, done, info = env.step(action)
    prev_obs = new_observation
    game_memory.append([new_observation, action])
    score += reward
    print(step_index)
    if done:
      break
  
  env.reset()
  scores.append(score)

print(scores)
print('Average Score:',sum(scores)/len(scores))
print('choice 1:{}  choice 0:{} choice 2:{}'.format(choices.count(1)/len(choices),choices.count(0)/len(choices),choices.count(2)/len(choices)))

In [22]:
#Sometimes the order of files can be different.
import os
from PIL import Image
images = []
filelist = os.listdir('/content/smartcar/')
filelist = sorted(filelist,key=lambda x: int(os.path.splitext(x)[0]))

for file in filelist:
  img = Image.open('/content/smartcar/' + file)
  images.append(img)
images[0].save('smartcar.gif',save_all=True, append_images=images[1:], optimize=False, loop = True,duration = 150)